In [1]:
# import cvxopt
# import cvxopt.glpk
# from cvxopt import matrix, solvers
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
import datetime
import seaborn as sns
import itertools
from scipy.linalg import block_diag
import time
import os
import pickle
from scipy.misc import comb
from itertools import combinations
from functools import partial

In [2]:
from __future__ import print_function

import sys

import cplex
from cplex.exceptions import CplexError

In [3]:
profile = np.array([list(np.random.permutation(range(1,11))) for i in range(5)])

In [179]:
stringmatrix = '['
for i in range(profile.shape[0]):
    for j in range(profile.shape[1]):
        if j!=profile.shape[1]-1:
            stringmatrix+=str(profile[i][j])+','
        else:
            stringmatrix+=str(profile[i][j])+';'
stringmatrix=stringmatrix[:-1]
stringmatrix+=']'
print (stringmatrix)

[2,4,3,1,9,7,8,6,5,10;10,4,1,8,5,9,7,2,3,6;2,3,8,10,7,1,4,9,5,6;10,5,4,7,9,2,1,8,6,3;7,1,4,2,9,6,8,10,3,5]


In [5]:
for p in [1,2,3]:
    for typ in ['rate','sort']:
        with open('./q{}_{}_profile.p'.format(p,typ),'rb') as file_:
            profile = pickle.load(file_).astype(np.int64)
            print(profile)
            n,m = len(profile), len(profile[0])
            # print(n, m)

            k=4

            def Ind(m,k,i,r):
                return (i-1)*(m-k+1)+r+m
            IndY = partial(Ind,m,k)
            # IndY(1,1)

            nIneq  = int(n*((m-k+2)*(m-k+3)/2-1)+comb(m,k))
            nEq = 1
            nVar = int(m+n*(m-k+1)+1)
            Aineq = np.zeros((nIneq, nVar))
            bineq = np.zeros((nIneq, 1))
            Aeq = np.zeros((nEq, nVar))
            beq = np.zeros((nEq, 1))
            f = np.zeros((nVar, 1))
            ctype = 'B'*(nVar-1)+'C'
            f[-1] = 1
            Aeq[0][0:m] = 1
            beq = k
            countIneq = 1

            for i in range(1,n+1):
                for r in range(1,m-k+2):
                    for j in range(1,r+1):
                        Aineq[countIneq-1, IndY(i,r)-1] = 1
            #             print(IndY(i,r))
            #             print(Aineq)
                        Aineq[countIneq-1, profile[i-1,j-1]-1] = 1
            #             print (Aineq)
                        bineq[countIneq-1] = 1
                        countIneq = countIneq + 1
                    Aineq[countIneq-1, IndY(i,r)-1] = -1
                    for j in range(1,r+1):
                        Aineq[countIneq-1, profile[i-1,j-1]-1] = -1
                    bineq[countIneq-1] = -1
                    countIneq = countIneq + 1
            list_T = np.array([list(int(y)+1 for y in x) for x in list(combinations(range(m),k))])
            for indT in range(1, list_T.shape[0]+1):
                T = list_T[indT-1]
                first_indices_T = np.array([[i for i,elem in enumerate(x) if x[i] in T][0] + 1 for x in profile])
                Aineq[countIneq-1, nVar-1] = -1
                for i in range(1,n+1):
                    Aineq[countIneq-1,IndY(i,first_indices_T[i-1])-1] = 1./first_indices_T[i-1]
                countIneq = countIneq + 1

            # data common to all populateby functions
            my_obj = list(i[0] for i in f)
            my_ub = [cplex.infinity]*len(f)
            my_lb = [0.0]*len(f)
            my_ctype = ctype
            my_colnames = ['x{}'.format(i) for i in range(len(f))]
            my_rhs = list(i[0] for i in bineq) + [float(beq)]
            my_rownames = ['r{}'.format(i) for i in range(nIneq+nEq)]
            my_sense = 'L'*nIneq+'E'*nEq
        
            def populatebynonzero(prob):
                prob.objective.set_sense(prob.objective.sense.minimize)

                prob.linear_constraints.add(rhs=my_rhs, senses=my_sense,
                                            names=my_rownames)
                prob.variables.add(obj=my_obj, lb=my_lb, ub=my_ub, types=my_ctype,
                                   names=my_colnames)

                rows, cols = np.nonzero(np.concatenate((Aineq,Aeq)))
                rows=list(rows)
                cols=list(cols)
                vals = [np.concatenate((Aineq,Aeq))[rows[i],cols[i]] for i in range(len(rows))]

                prob.linear_constraints.set_coefficients(zip(rows, cols, vals))


            def mipex1(pop_method):

                try:
                    my_prob = cplex.Cplex()

                    handle = populatebynonzero(my_prob)

                    my_prob.solve()
                except CplexError as exc:
                    print(exc)
                    return

                print()
                # solution.get_status() returns an integer code
                print("Solution status = ", my_prob.solution.get_status(), ":", end=' ')
                # the following line prints the corresponding string
                print(my_prob.solution.status[my_prob.solution.get_status()])
                print("Solution value  = ", my_prob.solution.get_objective_value())

                numcols = my_prob.variables.get_num()
                numrows = my_prob.linear_constraints.get_num()

                slack = my_prob.solution.get_linear_slacks()
                x = my_prob.solution.get_values()

            #     for j in range(numrows):
            #         print("Row %d:  Slack = %10f" % (j, slack[j]))
                for j in range(m):
                    print("Column %d:  Value = %10f" % (j, x[j]))
                print ("Column %d:  Value = %10f" % (numcols-1, x[numcols-1]))
                return x[:10]

            choices = mipex1('n')
            print (list((np.array(np.where(np.array(choices)>0.0))+1)[0]))
            selected_points = list((np.array(np.where(np.array(choices)>0.5))+1)[0])
            with open('./q{}_{}_results.p'.format(p,typ),'wb') as file_:
                pickle.dump(selected_points, file_)

[[ 7  4  9  8  3  2  6  1 10  5]
 [ 1  9  3  2  8  7  6 10  4  5]
 [ 1  8  4  5  2 10  7  6  3  9]
 [ 7  2  6  1  9  5  8  3  4 10]
 [ 8  5  1 10  7  2  6  9  4  3]
 [ 2  9  7  3  1 10  6  4  5  8]
 [10  8  5  2  4  3  9  7  1  6]
 [ 4  8  1 10  6  7  9  5  3  2]
 [ 8  9 10  1  5  4  3  2  7  6]
 [ 1 10  7  4  2  3  6  9  5  8]
 [ 2  8  4  7  3  9  5 10  6  1]
 [ 8  9  5  3 10  1  2  7  6  4]
 [ 3  2  9  1  7  4  6 10  8  5]
 [ 3 10  2  6  9  5  7  4  8  1]
 [ 7  5  9  1  8  6 10  2  4  3]
 [ 1  2  9  7  6  8  5  3  4 10]]
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Read_APIEncoding                        "UTF-8"
CPXPARAM_MIP_Strategy_CallbackReducedLP          0


Tried aggregator 2 times.
MIP Presolve eliminated 19 rows and 0 columns.
Aggregator did 16 substitutions.
Reduced MIP has 736 rows, 107 columns, and 4179 nonzeros.
Reduced MIP has 106 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (5.15 ticks)
Found incumbent of value 4.000000 after 0.02 sec. (5.75 ticks)
Probing fixed 16 vars, tightened 0 bounds.
Probing time = 0.02 sec. (10.26 ticks)
Cover probing fixed 0 vars, tightened 11 bounds.
Tried aggregator 1 time.
MIP Presolve eliminated 523 rows and 72 columns.
MIP Presolve modified 277 coefficients.
Reduced MIP has 213 rows, 35 columns, and 1254 nonzeros.
Reduced MIP has 34 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (4.64 ticks)
Probing fixed 30 vars, tightened 1 bounds.
Probing changed sense of 2 constraints.
Probing time = 0.00 sec. (0.32 ticks)
Cover probing fixed 0 vars, tightened 105 bounds.
Tried aggregator 1 time.
MIP Presolve eliminated 213 rows and 35 columns.
All rows and 

Tried aggregator 2 times.
MIP Presolve eliminated 25 rows and 0 columns.
Aggregator did 14 substitutions.
Reduced MIP has 662 rows, 95 columns, and 3647 nonzeros.
Reduced MIP has 94 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (4.43 ticks)
Found incumbent of value 6.000000 after 0.02 sec. (4.96 ticks)
Probing fixed 14 vars, tightened 0 bounds.
Probing time = 0.01 sec. (8.47 ticks)
Cover probing fixed 0 vars, tightened 9 bounds.
Tried aggregator 1 time.
MIP Presolve eliminated 183 rows and 27 columns.
Reduced MIP has 479 rows, 68 columns, and 3056 nonzeros.
Reduced MIP has 67 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (4.01 ticks)
Probing fixed 28 vars, tightened 0 bounds.
Probing time = 0.01 sec. (4.56 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 185 rows and 30 columns.
MIP Presolve modified 4 coefficients.
Reduced MIP has 294 rows, 38 columns, and 1885 nonzeros.
Reduced MIP has 37 binaries, 0 generals, 0 SOSs, and

Tried aggregator 2 times.
MIP Presolve eliminated 9 rows and 0 columns.
Aggregator did 9 substitutions.
Reduced MIP has 508 rows, 65 columns, and 2809 nonzeros.
Reduced MIP has 64 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (2.81 ticks)
Found incumbent of value 6.833333 after 0.02 sec. (5.18 ticks)
Probing fixed 9 vars, tightened 0 bounds.
Probing time = 0.01 sec. (4.80 ticks)
Cover probing fixed 0 vars, tightened 1 bounds.
Tried aggregator 1 time.
MIP Presolve eliminated 76 rows and 10 columns.
Reduced MIP has 432 rows, 55 columns, and 2592 nonzeros.
Reduced MIP has 54 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (3.14 ticks)
Probing time = 0.00 sec. (2.28 ticks)
Tried aggregator 1 time.
Reduced MIP has 432 rows, 55 columns, and 2592 nonzeros.
Reduced MIP has 54 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (1.66 ticks)
Probing time = 0.00 sec. (2.28 ticks)
Clique table members: 648.
MIP emphasis: b

Tried aggregator 2 times.
MIP Presolve eliminated 20 rows and 0 columns.
Aggregator did 12 substitutions.
Reduced MIP has 599 rows, 83 columns, and 3279 nonzeros.
Reduced MIP has 82 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (3.72 ticks)
Found incumbent of value 10.333333 after 0.02 sec. (7.03 ticks)
Probing fixed 12 vars, tightened 0 bounds.
Probing time = 0.01 sec. (6.47 ticks)
Cover probing fixed 0 vars, tightened 2 bounds.
Tried aggregator 1 time.
MIP Presolve eliminated 100 rows and 13 columns.
Reduced MIP has 499 rows, 70 columns, and 2931 nonzeros.
Reduced MIP has 69 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (3.78 ticks)
Probing time = 0.00 sec. (2.74 ticks)
Tried aggregator 1 time.
Reduced MIP has 499 rows, 70 columns, and 2931 nonzeros.
Reduced MIP has 69 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (1.98 ticks)
Probing time = 0.00 sec. (2.73 ticks)
Clique table members: 1061.
MIP empha

Tried aggregator 2 times.
MIP Presolve eliminated 24 rows and 0 columns.
Aggregator did 12 substitutions.
Reduced MIP has 595 rows, 83 columns, and 3233 nonzeros.
Reduced MIP has 82 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (3.72 ticks)
Found incumbent of value 4.000000 after 0.01 sec. (4.20 ticks)
Probing fixed 12 vars, tightened 0 bounds.
Probing time = 0.01 sec. (6.61 ticks)
Cover probing fixed 0 vars, tightened 9 bounds.
Tried aggregator 1 time.
MIP Presolve eliminated 151 rows and 22 columns.
MIP Presolve modified 3 coefficients.
Reduced MIP has 444 rows, 61 columns, and 2779 nonzeros.
Reduced MIP has 60 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (3.52 ticks)
Probing fixed 43 vars, tightened 0 bounds.
Probing changed sense of 2 constraints.
Probing time = 0.00 sec. (2.69 ticks)
Cover probing fixed 0 vars, tightened 1 bounds.
Tried aggregator 2 times.
MIP Presolve eliminated 309 rows and 43 columns.
MIP Presolve modifie

Tried aggregator 2 times.
MIP Presolve eliminated 14 rows and 0 columns.
Aggregator did 11 substitutions.
Reduced MIP has 571 rows, 77 columns, and 3206 nonzeros.
Reduced MIP has 76 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (3.50 ticks)
Found incumbent of value 6.000000 after 0.01 sec. (3.97 ticks)
Probing fixed 11 vars, tightened 0 bounds.
Probing time = 0.01 sec. (6.30 ticks)
Cover probing fixed 0 vars, tightened 5 bounds.
Tried aggregator 1 time.
MIP Presolve eliminated 113 rows and 16 columns.
Reduced MIP has 458 rows, 61 columns, and 2885 nonzeros.
Reduced MIP has 60 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (3.66 ticks)
Probing fixed 7 vars, tightened 0 bounds.
Probing time = 0.01 sec. (3.85 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 49 rows and 8 columns.
Reduced MIP has 409 rows, 53 columns, and 2690 nonzeros.
Reduced MIP has 52 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. 

In [35]:
selected_points = list((np.array(np.where(np.array(choices)>0.0))+1)[0])
selected_points

[1, 2, 3, 8, 9]

In [197]:
matlab = np.loadtxt('../nisarg_code/testmatrix', delimiter=',')

In [198]:
print(matlab)

[[ 0.  1.  0. ...,  0.  0.  0.]
 [ 0. -1.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0. -1.]
 [ 0.  0.  0. ...,  0.  0. -1.]
 [ 0.  0.  0. ...,  0.  0. -1.]]


In [226]:
print(Aineq)

[[ 0.  1.  0. ...,  0.  0.  0.]
 [ 0. -1.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0. -1.]
 [ 0.  0.  0. ...,  0.  0. -1.]
 [ 0.  0.  0. ...,  0.  0. -1.]]


In [49]:
with open('./q2_sort_profile.p','rb') as file_:
    profile = pickle.load(file_)
profile.astype(np.int64)

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [10, 10, 10,  0,  0,  0,  0,  0, 10, 10],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [10, 10,  0,  0, 10, 10,  0,  0, 10,  0],
       [10,  0,  0,  0, 10,  0,  0, 10, 10, 10],
       [ 0,  0,  0,  0, 10, 10, 10, 10, 10,  0],
       [ 0,  0,  0,  0, 10, 10, 10, 10, 10,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [10, 10, 10,  0,  0,  0, 10,  0, 10,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])

In [56]:
p

NameError: name 'p' is not defined